###  importing relevant libraries

In [2]:
from scapy.all import*
import math
import pandas as pd
import os
import numpy as np
import zipfile
import os
import shutil

In [3]:
path="./pcaps/"

In [4]:
def find_the_way(path,file_format,con=""):
    files_add = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if file_format in file:
                if con in file:
                    files_add.append(os.path.join(r, file))  
            
    return files_add
files_add=find_the_way('./pcaps','.pcap')
files_add

['./pcaps\\Chime_Doorbell\\2019-06-29.pcap',
 './pcaps\\Chime_Doorbell\\2019-07-01.pcap',
 './pcaps\\Chime_Doorbell\\2019-07-02.pcap',
 './pcaps\\Chime_Doorbell\\2019-07-03.pcap',
 './pcaps\\Chime_Doorbell\\2019-07-04.pcap',
 './pcaps\\Chime_Doorbell\\2019-07-05.pcap',
 './pcaps\\Chime_Doorbell\\2019-07-06.pcap',
 './pcaps\\Chime_Doorbell\\2019-07-07.pcap',
 './pcaps\\Chime_Doorbell\\2019-07-08.pcap',
 './pcaps\\Chime_Doorbell\\2019-07-09.pcap',
 './pcaps\\Chime_Doorbell\\2019-07-10.pcap',
 './pcaps\\Chime_Doorbell\\2019-07-11.pcap',
 './pcaps\\Chime_Doorbell\\2019-07-12.pcap',
 './pcaps\\Chime_Doorbell\\2019-07-13.pcap',
 './pcaps\\Chime_Doorbell\\2019-07-14.pcap',
 './pcaps\\Chime_Doorbell\\2019-07-15.pcap',
 './pcaps\\Chime_Doorbell\\2019-07-16.pcap',
 './pcaps\\Chime_Doorbell\\2019-07-17.pcap',
 './pcaps\\Chime_Doorbell\\2019-07-18.pcap',
 './pcaps\\Chime_Doorbell\\2019-07-19.pcap',
 './pcaps\\D-Link_Cam936L\\2019-06-29.pcap',
 './pcaps\\D-Link_Cam936L\\2019-07-01.pcap',
 './pcaps\

### Port numbers are classified in this part as:

| Port Numbers | Equivalents |
| :------ | :------ |
|No port| 0|
|Well known ports (between 0 and 1023) |1|
|Rregistered ports (between 1024 and 49151)  |2|
|Dynamic ports (between  49152 and 65535) |3|
# ↓ 

In [5]:
def port_class(port):
    if 0 <= port <= 1023:
        return 1
    elif  1024 <= port <= 49151 :
        return 2
    elif 49152 <=port <= 65535 :
        return 3
    else:
        return 0


### The dictionary to be used for MAC address and device matching.
#### Datasets, their MAC addresses and Devices are given separately.

In [6]:
MAC_list={":\\xe8\\x13ua\\x00\\x00'":['2c:3a:e8:13:75:61', 'oossxx_SmartPlug'],
          "b',:\\xe8\\x13ua\\x00\\x00'":['2c:3a:e8:13:75:61', 'oossxx_SmartPlug'],
 "b'$\\xf5\\xa2\\xff\\xad\\x97\\x00\\x00'":['24:f5:a2:ff:ad:97', 'Wemo_SmartPlug'],
 "b'(\\xad>r\\xbd~\\x00\\x00'":['28:ad:3e:72:bd:7e', 'Wans_Cam'],
 "b'4\\x03\\xde\\x14\\xdc;\\x00\\x00'":['34:03:de:14:dc:3b', 'Ring_Doorbell'],
 "b'P\\xc7\\xbf\\xeb\\xec\\xc4\\x00\\x00'":[ '50:c7:bf:eb:ec:c4', 'tp-link_LightBulb'],
 "b'\\x00\\xe0Lq\\xb6\\xb6\\x00\\x00'":[ '00:e0:4c:71:b6:b6', 'Minger_LightStrip'],
 "b'\\x0c\\x80ced\\xde\\x00\\x00'": ['0c:80:63:65:64:de', 'tp-link_SmartPlug'],
 "b'\\x14k\\x9c\\x85\\x8a\\xfe\\x00\\x00'":[ '14:6b:9c:85:8a:fe', 'Tenvis_Cam'],
 "b'\\x84 \\x96\\x90C\\xe6\\x00\\x00'":[ '84:20:96:90:43:e6', 'itTiot_Cam'],
 "b'\\x84\\r\\x8e@\\x04T\\x00\\x00'": ['84:0d:8e:40:04:54', 'Lumiman_SmartPlug'] ,
 "b'\\x84\\r\\x8e\\x80b\\x9a\\x00\\x00'":['84:0d:8e:80:62:9a', 'Smart_LightStrip'],
 "b'\\x84\\xf3\\xeb<\\x9eO\\x00\\x00'":[ '84:f3:eb:3c:9e:4f', 'LaCrosse_AlarmClock'],
 "b'\\x84\\xf3\\xeb\\x18\\x91\\xdd\\x00\\x00'":[ '84:f3:eb:18:91:dd', 'Goumia_Coffemaker'],
 "b'\\x84\\xf3\\xebwH\\x1f\\x00\\x00'":['84:f3:eb:77:48:1f', 'Gosuna_LightBulb'],
 "b'\\x88?J\\xf5\\xde\\xd0\\x00\\x00'":['88:3f:4a:f5:de:d0', 'Chime_Doorbell'],
 "b'\\xb2\\xc5TBy\\xf2\\x00\\x00'":['b2:c5:54:42:79:f2', 'D-Link_Cam936L'],
 "b'\\xbc\\xdd\\xc2\\xe1%)\\x00\\x00'":[ 'bc:dd:c2:e1:25:29', 'Smart_Lamp'],
 'b\'\\xdcO"\\x1b/\\xad\\x00\\x00\'':['dc:4f:22:1b:2f:ad', 'Lumiman_Bulb600'],
 "b'\\xec=\\xfdZ\\xbay\\x00\\x00'":[ 'ec:3d:fd:5a:ba:79', 'Ocean_Radio'],
 "b'`8\\xe0\\x12\\x83\\x8a\\x00\\x00'":["UNK","UNK"],
          "b'\\x00\\x13\\xef\\x83\\x00M\\x00\\x00'":["UNK","UNK"],
          ":\\xe8\\x13ua\\x00\\x00'":["UNK","UNK"],

 "b'`\\x01\\x94\\xa7\\xc9\\xfd\\x00\\x00'": ['60:01:94:a7:c9:fd', 'Renpho_SmartPlug'],
 "b'`\\x01\\x94\\xb6\\x1b)\\x00\\x00'":[ '60:01:94:b6:1b:29', 'Gosuna_Socket'],
 "b'`\\x01\\x94n?p\\x00\\x00'":['60:01:94:6e:3f:70', 'Lumiman_Bulb900']}

### Calculating the payload entropy value.


# ↓ 

In [7]:
def pre_entropy(payload):
    
    characters=[]
    for i in payload:
            characters.append(i)
    return shannon(characters)


def shannon(data):
    freq_dict={} 
    for i in data:
        if i in freq_dict:
            freq_dict[i] += 1
        else:
            freq_dict[i] = 1    
    entropy = 0.0
    logarithm_base = 2
    payload_size = len(data) #
    for key in freq_dict.keys():
        frequency = float(freq_dict[key])/payload_size
        if frequency > 0: 
            entropy = entropy + frequency * math.log(frequency, logarithm_base)
    return -entropy

In [8]:
count=0


import time

for numero,i in enumerate (files_add):
    dst_ip_list={}
    for mac in MAC_list:
        dst_ip_list[mac]=[]
    filename=i[:-4]+"_IOT.csv"
    ths = open(filename, "w")
    header="ARP,LLC,EAPOL,IP,ICMP,ICMP6,TCP,UDP,TCP_w_size,HTTP,HTTPS,DHCP,BOOTP,SSDP,DNS,MDNS,NTP,IP_padding,IP_add_count,IP_ralert,Portcl_src,Portcl_dst,Pck_size,Pck_rawdata,payload_l,Entropy,MAC,Label\n"
    ths.write(header)  
    
    
    pkt = rdpcap(i)
    sayaç=len(pkt)//20
    print(f"       -{numero}/{len(files_add)}" ,)
    for jj,j in enumerate(pkt):
        
        try:
            
            if jj%sayaç==0:
                sys.stdout.write("\r[" + "=" * int(jj//sayaç) +  " " * int((sayaç*20 - jj)// sayaç) + "]" +  str(5*jj//sayaç) + "%")
                sys.stdout.flush()
        except:pass
        if j.haslayer(ARP):
            continue
        else:
        
        

            ip_add_count=0
            layer_2_arp = 0
            layer_2_llc = 0

            layer_3_eapol = 0        
            layer_3_ip = 0
            layer_3_icmp = 0
            layer_3_icmp6 = 0



            layer_4_tcp = 0
            layer_4_udp = 0
            layer_4_tcp_ws=0


            layer_7_http = 0
            layer_7_https = 0
            layer_7_dhcp = 0
            layer_7_bootp = 0
            layer_7_ssdp = 0
            layer_7_dns = 0
            layer_7_mdns = 0
            layer_7_ntp = 0

            ip_padding = 0
            ip_ralert = 0


            port_class_src = 0
            port_class_dst = 0

            pck_size = 0
            pck_rawdata = 0
            entropy=0


            layer_4_payload_l=0




            try:
                pck_size=j.len
            except:pass

            try:
                if j[IP]:
                    layer_3_ip = 1     
                temp=str(j[IP].dst)
                if temp not in dst_ip_list[j.src]:
                    dst_ip_list[j.src].append(temp)
                ip_add_count=len(dst_ip_list[j.src])
                port_class_src = port_class(j[IP].sport)
                port_class_dst = port_class(j[IP].dport)

            except:pass 

            temp=str(j.show)
            if "ICMPv6" in temp:
                layer_3_icmp6 = 1

            try:
                if j[IP].ihl >5:
                    if IPOption_Router_Alert(j):
                        pad=str(IPOption_Router_Alert(j).show)
                        if "Padding" in pad:
                            ip_padding=1
                        ip_ralert = 1     
            except:pass 

            if j.haslayer(ICMP):
                layer_3_icmp = 1  


            if j.haslayer(Raw):
                pck_rawdata = 1   

            if j.haslayer(UDP):

                layer_4_udp = 1
                if j[UDP].sport==68 or j[UDP].sport==67:
                    layer_7_dhcp = 1
                    layer_7_bootp = 1
                if j[UDP].sport==53 or j[UDP].dport==53:
                    layer_7_dns = 1      
                if j[UDP].sport==5353 or j[UDP].dport==5353:
                    layer_7_mdns = 1                    
                if j[UDP].sport==1900 or j[UDP].dport==1900:
                    layer_7_ssdp = 1                    
                if j[UDP].sport==123 or j[UDP].dport==123:
                    layer_7_ntp = 1                    

            try:
                if j[UDP].payload:
                    layer_4_payload_l=len(j[UDP].payload)
            except:pass  



            if j.haslayer(TCP):
                layer_4_tcp = 1
                layer_4_tcp_ws=j[TCP].window
                if j[TCP].sport==80 or j[TCP].dport==80:
                    layer_7_http = 1      
                if j[TCP].sport==443 or j[TCP].dport==443:
                    layer_7_https = 1  
                try:
                    if j[TCP].payload:
                        layer_4_payload_l=len(j[TCP].payload)
                except:pass  

            if j.haslayer(ARP):
                layer_2_arp = 1                                 

            if j.haslayer(LLC):
                layer_2_llc = 1                             

            if j.haslayer(EAPOL):        
                layer_3_eapol = 1                                 
            pdata=[]
            if "TCP" in j:            
                pdata = (j[TCP].payload)
            if "Raw" in j:
                pdata = (j[Raw].load)
            elif "UDP" in j:            
                pdata = (j[UDP].payload)
            elif "ICMP" in j:            
                pdata = (j[ICMP].payload)
            pdata=list(memoryview(bytes(pdata)))            

            if pdata!=[]:
                entropy=shannon(pdata)        
            else:
                entropy=0
            label=MAC_list[str(j.src)][1]
            clean_mac=MAC_list[str(j.src)][0]
            line=[layer_2_arp, layer_2_llc, layer_3_eapol, layer_3_ip, layer_3_icmp, layer_3_icmp6, layer_4_tcp, layer_4_udp, layer_4_tcp_ws, layer_7_http, layer_7_https, layer_7_dhcp, layer_7_bootp, layer_7_ssdp, layer_7_dns, layer_7_mdns, layer_7_ntp, ip_padding, ip_add_count, ip_ralert, port_class_src, port_class_dst, pck_size, pck_rawdata,layer_4_payload_l,entropy, clean_mac,label]  
            line=str(line).replace("[","")
            line=str(line).replace("]","")
            line=str(line).replace(", ",",")
            line=str(line).replace("\'","")
            if label!="unknown":
                ths.write(str(line)+"\n")  

    ths.close()          


       -0/440
[====================]100%       -1/440
[====================]100%       -2/440
[====================]100%       -3/440
[====================]100%       -4/440
[====================]100%       -5/440
[====================]100%       -6/440
[====================]100%       -7/440
[====================]100%       -8/440
[====================]100%       -9/440
[====================]100%       -10/440
[====================]100%       -11/440
[====================]100%       -12/440
[====================]100%       -13/440
[====================]100%       -14/440
[====================]100%       -15/440
[====================]100%       -16/440
[====================]100%       -17/440
[====================]100%       -18/440
[====================]100%       -19/440
[====================]100%       -20/440
[====================]100%       -21/440
[====================]100%       -22/440
[====================]100%       -23/440
[====================]100%       -24/440
[==========

[====================]100%       -198/440
[====================]100%       -199/440
[=================== ]95%       -200/440
[====================]100%       -201/440
[====================]100%       -202/440
[====================]100%       -203/440
[====================]100%       -204/440
[====================]100%       -205/440
[====================]100%       -206/440
[====================]100%       -207/440
[====================]100%       -208/440
[=================== ]95%       -209/440
[====================]100%       -210/440
[====================]100%       -211/440
[====================]100%       -212/440
[====================]100%       -213/440
[====================]100%       -214/440
[====================]100%       -215/440
[====================]100%       -216/440
[====================]100%       -217/440
[====================]100%       -218/440
[====================]100%       -219/440
[====================]100%       -220/440
[====================]100%       -22

[=================== ]95%       -393/440
[====================]100%       -394/440
[====================]100%       -395/440
[====================]100%       -396/440
[====================]100%       -397/440
[====================]100%       -398/440
[====================]100%       -399/440
[====================]100%       -400/440
[====================]100%       -401/440
[====================]100%       -402/440
[====================]100%       -403/440
[====================]100%       -404/440
[====================]100%       -405/440
[====================]100%       -406/440
[====================]100%       -407/440
[====================]100%       -408/440
[====================]100%       -409/440
[====================]100%       -410/440
[====================]100%       -411/440
[====================]100%       -412/440
[====================]100%       -413/440
[====================]100%       -414/440
[====================]100%       -415/440
[====================]100%       -4